In [ ]:
#!pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

In [ ]:
# driver = webdriver.Chrome()
# driver.get('https://www.wildberries.ru/')

In [ ]:
from selenium.webdriver.chrome.options import Options

options = Options()
# запуск на полный экран
options.add_argument('start-maximized')

driver = webdriver.Chrome(options=options)

driver.get('https://www.wildberries.ru/')
time.sleep(3)

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

input = driver.find_element(By.ID, 'searchInput')
input.send_keys('гелевые стельки')
input.send_keys(Keys.ENTER)
time.sleep(2)

Так как процесс парсинга состоит из 3 операций: скролинг (прокрутка страницы), записи данных и пагинации (переход между страницами), выделим эти действия в отдельные функции

In [ ]:
def scrolling(page):
    while True:
        # будем ждать пока загрузятся все карточки
        wait = WebDriverWait(driver, timeout=20)
        # ищем карточки
        cards = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="product-card-list"]/article')))
        # определяем их количество
        count = len(cards)
        # выполнение кода из JavaScript
        driver.execute_script('window.scrollBy(0,2000)')    # прокрутка вниз на 1000 пикселей
        
        time.sleep(1)
        # вполняем подсчет карточек после скроллинга
        cards = driver.find_elements(By.XPATH, '//div[@class="product-card-list"]/article')
        
        # если кол-во карточек не меняется, выходим из цикла
        if len(cards) == count:
            break
    print(f'Скролинг {page} страницы завершен')
    return cards

In [ ]:
def pagination():
    
    button = driver.find_element(By.XPATH, '//a[@class="pagination-next pagination__next j-next-page"]')
    actions = ActionChains(driver)
    actions.move_to_element(button).click()
    actions.perform()
    

In [ ]:
def value_addition(values, list_result):
    for value in values:
        price = value.find_element(By.CLASS_NAME, 'price__lower-price').text
        name = value.find_element(By.XPATH, './/div/a').get_attribute('aria-label')
        url = value.find_element(By.XPATH, './/div/a').get_attribute('href')
        list_result.append({
            'name': name,
            'url': url,
            'price': price
        })

In [ ]:
card_list = []
page = 1

while True:
    cards = scrolling(page)
    time.sleep(1)
    if len(cards) > 1:
        value_addition(cards, card_list)
        print(f'Данные {page} страницы записаны')
        time.sleep(0.5)
        page += 1
    else:
        driver.refresh()
        print(f'Страницу {page} пришлось обновить')
        time.sleep(2)
        continue
    try:
        pagination()
        print(f'Перешли на {page} страницу')
        time.sleep(0.5)
    except Exception as e:
        print(e)
        break



In [ ]:
from pprint import pprint
pprint(card_list)

In [ ]:
len(card_list)

In [ ]:
import pandas as pd

df = pd.DataFrame(card_list)
df

In [ ]:
df.to_csv('result.csv', index=False)